1. Choose a target variable from the dataset.

2. Determine whether the selected column's data can be used to solve a regression or classification problem.

3. Begin by preparing the dataset, separating it into features (X) and labels (Y).

4. Address missing data by either removing rows or filling in the gaps.

5. Utilize dummy hot encoding to transform non-numeric data into a numeric format.

6. Assemble the feature set (X) to include all relevant features, while the label set (Y) contains the values from the chosen column.

In [1]:
from google.colab import drive
drive.mount('Indicadores_municipales_sabana_DA')

Mounted at Indicadores_municipales_sabana_DA


In [2]:
import pandas as pd

# Cargar un archivo CSV en un DataFrame especificando la codificación
df = pd.read_csv('Indicadores_municipales_sabana_DA.csv', encoding='latin1')

In [3]:
# Mostrar las primeras filas del DataFrame para verificar que se haya cargado correctamente
df.head()

ent         nom_ent  mun  clave_mun         nom_mun  pobtot_ajustada  \
0    1  Aguascalientes    1       1001  Aguascalientes           794304   
1    1  Aguascalientes    2       1002        Asientos            48592   
2    1  Aguascalientes    3       1003        Calvillo            53104   
3    1  Aguascalientes    4       1004           Cosío            14101   
4    1  Aguascalientes    5       1005     Jesús María           101379   

     pobreza  pobreza_e  pobreza_m    vul_car  ...  pobreza_alim_10  \
0  30.531104   2.264478  28.266627  27.983320  ...        11.805700   
1  67.111172   8.040704  59.070468  22.439389  ...        21.993299   
2  61.360527   7.241238  54.119289  29.428583  ...        19.266800   
3  52.800458   4.769001  48.031458  27.128568  ...        14.303200   
4  45.338512   6.084037  39.254475  26.262912  ...        15.085100   

   pobreza_cap_90  pobreza_cap_00  pobreza_cap_10  pobreza_patrim_90  \
0            20.4            12.7       18.474600               43.4   
1            39.9            29.0       30.980801               64.2   
2            39.5            33.1       28.259199               63.9   
3            35.2            21.0       22.386101               59.7   
4            36.6            22.6       22.139999               60.6   

   pobreza_patrim_00  pobreza_patrim_10  gini_90  gini_00   gini_10  
0               33.7          41.900398    0.473    0.425  0.422628  
1               48.9          59.175800    0.379    0.533  0.343879  
2               57.9          56.504902    0.414    0.465  0.386781  
3               40.1          51.164501    0.392    0.541  0.344984  
4               42.2          45.703899    0.391    0.469  0.458083  

[5 rows x 139 columns]

Divide my features into 'y' and 'x'

In [4]:
columns_to_exclude = ["ent", "nom_ent", "mun","nom_mun","N_ic_rezedu"]
X = df.drop(columns_to_exclude, axis=1)

y = df["N_ic_rezedu"]

print("data already divided")
print(X.columns)

data already divided
Index(['clave_mun', 'pobtot_ajustada', 'pobreza', 'pobreza_e', 'pobreza_m',
       'vul_car', 'vul_ing', 'npnv', 'ic_rezedu', 'ic_asalud',
       ...
       'pobreza_alim_10', 'pobreza_cap_90', 'pobreza_cap_00', 'pobreza_cap_10',
       'pobreza_patrim_90', 'pobreza_patrim_00', 'pobreza_patrim_10',
       'gini_90', 'gini_00', 'gini_10'],
      dtype='object', length=134)


We transformed the columns containing string values into a coded format and addressed missing values (NaN) within the dataset.

In [6]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer

label_encoder = LabelEncoder()
X['gdo_rezsoc00'] = label_encoder.fit_transform(X['gdo_rezsoc00'])
X['gdo_rezsoc05'] = label_encoder.fit_transform(X['gdo_rezsoc05'])
X['gdo_rezsoc10'] = label_encoder.fit_transform(X['gdo_rezsoc10'])

imputer = SimpleImputer(strategy="mean")

X_imputed = imputer.fit_transform(X)


scaler = StandardScaler()
X_scaler = scaler.fit_transform(X_imputed)

Data is divided into 80% for train an 20% for test.

In [7]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X_scaler, y, test_size=0.2, random_state=42)

We have imported a regression model for the solution.

In [8]:
from sklearn.linear_model import LinearRegression

modelo_regresion = LinearRegression()

The model has been trained.

In [9]:
modelo_regresion.fit(X_train, y_train)

LinearRegression()

The model predict the number of People with a lack of access to healthcare services

In [10]:
y_pred = modelo_regresion.predict(X_test)

We display the prediction alongside the corresponding municipality, and then we compare it with the actual value in the CSV.

In [11]:
X_test_df = pd.DataFrame(X_test, columns=X.columns)
codigos_municipio = X_test_df['clave_mun']



if len(y_pred) == len(y_test):
    for i in range(len(y_pred)):
        codigo_municipio = codigos_municipio.iloc[i]
        prediccion_pobreza = y_pred[i]
        pobreza_actual = y_test.iloc[i]
        print(f"For the municipality with the code {codigo_municipio}, It is predicted that there will be {prediccion_pobreza}  People with a lack of access to healthcare services. The actual value of 'N_ic_asalud' is {pobreza_actual}.")
else:
    print("The dimensions of y_pred and y_test do not match.")

For the municipality with the code 0.12838562200511897, It is predicted that there will be 2802.4240320780837  People with a lack of access to healthcare services. The actual value of 'N_ic_asalud' is 3045.
For the municipality with the code -0.7168338294212445, It is predicted that there will be 11518.155567178466  People with a lack of access to healthcare services. The actual value of 'N_ic_asalud' is 11963.
For the municipality with the code 0.6354357735439694, It is predicted that there will be 1142.0666486162572  People with a lack of access to healthcare services. The actual value of 'N_ic_asalud' is 2390.
For the municipality with the code -0.7128937291011409, It is predicted that there will be 149867.73460615403  People with a lack of access to healthcare services. The actual value of 'N_ic_asalud' is 163920.
For the municipality with the code -0.5792020492741772, It is predicted that there will be 109787.84784597302  People with a lack of access to healthcare services. The ac

We assess the model

In [12]:
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse}")
print(f"R^2: {r2}")

Mean Squared Error (MSE): 7063990.982293092
R^2: 0.9841521152540307
